In [3]:
from osgeo import gdal, osr
import numpy as np

def img_mosaic(path_1, path_2, path_out):
    # 打开Landsat-8影像
    ds1 = gdal.Open(path_1)
    # 打开Sentinel-2影像
    ds2 = gdal.Open(path_2)
    
    # 获取Landsat-8影像的投影信息，用作之后将Sentinel-2影像投影变换的目标投影
    proj_landsat_wkt = ds1.GetProjection()

    # 设置用于表示空值
    nodata_value = -9999

    # 创建一个内存数据集，用于存储重投影后的Sentinel-2数据
    driver = gdal.GetDriverByName('MEM')
    scale_factor = ds1.GetGeoTransform()[1] / ds2.GetGeoTransform()[1]  # 计算缩放比例，Landsat-8与Sentinel-2的分辨率比
    new_x_size = int(ds2.RasterXSize / scale_factor)  # 计算新的X方向像素大小
    new_y_size = int(ds2.RasterYSize / scale_factor)  # 计算新的Y方向像素大小
    ds2_reproj_resampled = driver.Create('11', new_x_size, new_y_size, ds2.RasterCount, gdal.GDT_Int16)

    # 设置新的地理变换参数
    new_geotransform = list(ds2.GetGeoTransform())
    new_geotransform[1] = ds1.GetGeoTransform()[1]  # 设置新的像素宽度
    new_geotransform[5] = -ds1.GetGeoTransform()[1]  # 设置新的像素高度
    ds2_reproj_resampled.SetGeoTransform(new_geotransform)
    ds2_reproj_resampled.SetProjection(proj_landsat_wkt)

    # 执行重投影和重采样操作
    gdal.ReprojectImage(ds2, ds2_reproj_resampled, ds2.GetProjection(), proj_landsat_wkt, gdal.GRA_Bilinear)

    # 为重投影和重采样后的Sentinel-2数据集设置NoData值
    for i in range(1, ds2_reproj_resampled.RasterCount + 1):
        ds2_reproj_resampled.GetRasterBand(i).SetNoDataValue(nodata_value)
        
    # 获取两个数据集的地理变换参数
    geo_trans_subs1 = ds1.GetGeoTransform()
    geo_trans_subs2 = ds2_reproj_resampled.GetGeoTransform()
    
    # 提取Landsat-8的红、绿、蓝和近红外波段数据
    dset_subs1 = [ds1.GetRasterBand(i).ReadAsArray() for i in [4, 3, 2, 5]]
    
    # 提取重采样后的Sentinel-2的红、绿、蓝和近红外波段数据
    dset_subs2 = [ds2_reproj_resampled.GetRasterBand(i).ReadAsArray() for i in [4, 3, 2, 8]]
    
    # 计算Landsat-8各波段的最小和最大值，排除NoData值
    min_vals_subs1 = [np.ma.masked_equal(band_data, nodata_value).min() for band_data in dset_subs1]
    max_vals_subs1 = [np.ma.masked_equal(band_data, nodata_value).max() for band_data in dset_subs1]
    
    # 计算Sentinel-2各波段的最小和最大值
    min_vals_subs2 = [band_data.min() for band_data in dset_subs2]
    max_vals_subs2 = [band_data.max() for band_data in dset_subs2]
    
    # 对Sentinel-2数据进行线性拉伸，以匹配Landsat-8的数据范围
    dset_subs2_stretched = []
    for i, band_data in enumerate(dset_subs2):
        stretched_band = ((band_data - min_vals_subs2[i]) / (max_vals_subs2[i] - min_vals_subs2[i])) * \
                         (max_vals_subs1[i] - min_vals_subs1[i]) + min_vals_subs1[i]
        stretched_band = np.clip(stretched_band, min_vals_subs1[i], None)  # 确保拉伸后的值不低于最小值
        stretched_band = stretched_band.astype(np.int16)  # 转换数据类型为整数
        dset_subs2_stretched.append(stretched_band)
    
    # 更新数据集
    dset_subs2 = dset_subs2_stretched

    # 计算影像拼接的分辨率和地理范围
    x_res_mosaic, y_res_mosaic = geo_trans_subs1[1], geo_trans_subs1[5]
    x_min_subs1 = geo_trans_subs1[0]
    y_max_subs1 = geo_trans_subs1[3]
    x_min_subs2 = geo_trans_subs2[0]
    y_max_subs2 = geo_trans_subs2[3]
    x_max_subs1 = x_min_subs1 + ds1.RasterXSize * geo_trans_subs1[1]
    y_min_subs1 = y_max_subs1 + ds1.RasterYSize * geo_trans_subs1[5]
    x_max_subs2 = x_min_subs2 + ds2_reproj_resampled.RasterXSize * geo_trans_subs2[1]
    y_min_subs2 = y_max_subs2 + ds2_reproj_resampled.RasterYSize * geo_trans_subs2[5]
    x_max_mosaic = max(x_max_subs1, x_max_subs2)
    x_min_mosaic = min(x_min_subs1, x_min_subs2)
    y_max_mosaic = max(y_max_subs1, y_max_subs2)
    y_min_mosaic = min(y_min_subs1, y_min_subs2)
    x_size_mosaic = int((x_max_mosaic - x_min_mosaic) / x_res_mosaic)
    y_size_mosaic = int(abs((y_max_mosaic - y_min_mosaic) / y_res_mosaic))
    
    # 设置拼接影像的地理变换信息
    geotrans_mosaic = [x_min_mosaic, x_res_mosaic, 0, y_max_mosaic, 0, y_res_mosaic]
    
    # 创建用于存储拼接后影像数据的数组，初始化为NoData值
    img_array_mosaic = np.full((len(dset_subs1), y_size_mosaic, x_size_mosaic), nodata_value)

    # 将Landsat-8数据填充到拼接影像数组中
    row_start_subs1 = int((y_max_subs1 - y_max_mosaic) / y_res_mosaic)
    col_start_subs1 = int((x_min_subs1 - x_min_mosaic) / x_res_mosaic)
    for i, band_data in enumerate(dset_subs1):
        img_array_mosaic[i, row_start_subs1:row_start_subs1 + ds1.RasterYSize, col_start_subs1:col_start_subs1 + ds1.RasterXSize] = band_data

    # 将重采样并拉伸后的Sentinel-2数据填充到拼接影像数组中
    row_start_subs2 = int((y_max_subs2 - y_max_mosaic) / y_res_mosaic)
    col_start_subs2 = int((x_min_subs2 - x_min_mosaic) / x_res_mosaic)
    for i, band_data in enumerate(dset_subs2):
        img_array_mosaic[i, row_start_subs2:row_start_subs2 + ds2_reproj_resampled.RasterYSize, col_start_subs2:col_start_subs2 + ds2_reproj_resampled.RasterXSize] = band_data
    
    # 创建输出影像文件
    driver = gdal.GetDriverByName('GTiff')
    dset_mosaic = driver.Create(path_out, xsize=x_size_mosaic, ysize=y_size_mosaic, bands=len(dset_subs1), eType=gdal.GDT_Int16)
    dset_mosaic.SetGeoTransform(geotrans_mosaic)
    dset_mosaic.SetProjection(ds1.GetProjection())

    # 将拼接影像的数据写入输出文件并设置NoData值
    for i in range(len(dset_subs1)):
        outband = dset_mosaic.GetRasterBand(i+1)
        outband.WriteArray(img_array_mosaic[i])
        outband.SetNoDataValue(nodata_value)

    # 关闭影像数据集，释放资源
    dset_mosaic = None

In [4]:
path_1 = 'data/Section-5/l8_chenggong_20200728.tif'
path_2 = 'data/Section-5/s2_kunming_chenggong_4bands_10m_subs1.tif'
path_out = 'data/Section-5/mosaic_test_4.tif'


##示例调用
img_mosaic(path_1=path_1, #定义第一幅待拼接影像的路径
           path_2=path_2, #定义第二幅待拼接影像的路径
           path_out=path_out #定义拼接影像的输出路径
           )


AttributeError: 'NoneType' object has no attribute 'SetGeoTransform'

number of fields(before field creating): 0
number of fields(after field creating): 3
